In [1]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
import core
import keras


PLOTS_DIR = f"plots/assignment_transformer"
MODEL_DIR = f"models/assignment_transformer"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)
plt.rcParams.update({"font.size": 14})

data_config = DataProcessor.load_from_npz(
    data_configs["data_path"]["nominal"], max_events=2_000_000
)
X_train, y_train,_,_ = DataProcessor.split_even_odd()
del DataProcessor

2025-12-02 10:44:46.998867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764668687.029539  579041 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764668687.039617  579041 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764668687.063947  579041 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764668687.063971  579041 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764668687.063973  579041 computation_placer.cc:177] computation placer alr

In [2]:
reload(Models)
reload(core)
TransformerMatcher = Models.FeatureConcatTransformer(
    data_config, name=r"Transformer"
)

# TransformerMatcher.load_model(f"{MODEL_DIR}/model.keras")
TransformerMatcher.build_model(
    hidden_dim=64,
    num_layers=6,
    num_heads=8,
    dropout_rate=0.1,
    compute_HLF=False,
)

TransformerMatcher.adapt_normalization_layers(X_train)

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
)

FeatureConcatTransformer is designed for classification tasks; regression targets will be ignored.


I0000 00:00:1764668830.599073  579041 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.
 50/472 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step    

I0000 00:00:1764668832.262046  582321 service.cc:152] XLA service 0x7f74e8004500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764668832.262074  582321 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-12-02 10:47:12.268135: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator pre_met_input_normalization_model_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-12-02 10:47:12.274534: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1764668832.287302  582321 cuda_dnn.cc:529] Loaded cuDNN version 91500
I0000 00:00:1764668832.405445  582321 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


472/472 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


2025-12-02 10:47:12.919849: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator pre_met_input_normalization_model_1/met_input_transform_1/assert_equal_1/Assert/Assert


Adapted normalization layer:  met_input_normalization
472/472 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Adapted normalization layer:  lep_input_normalization
472/472 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Adapted normalization layer:  jet_input_normalization


In [3]:
TransformerMatcher.train_model(
    epochs=100,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1,
        ),
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=15,
            restore_best_weights=True,
            verbose=1,
        ),

    ],
)

Epoch 1/100


2025-12-02 10:47:34.533009: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


777/781 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2541 - loss: 0.1520

2025-12-02 10:48:03.019215: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


781/781 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.2545 - loss: 0.1520

2025-12-02 10:48:19.780937: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-12-02 10:48:21.846783: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


781/781 ━━━━━━━━━━━━━━━━━━━━ 65s 42ms/step - accuracy: 0.2546 - loss: 0.1520 - val_accuracy: 0.5215 - val_loss: 0.1130 - learning_rate: 1.0000e-04
Epoch 2/100
781/781 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.4747 - loss: 0.1226 - val_accuracy: 0.6331 - val_loss: 0.0981 - learning_rate: 1.0000e-04
Epoch 3/100
781/781 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.5603 - loss: 0.1095 - val_accuracy: 0.6677 - val_loss: 0.0912 - learning_rate: 1.0000e-04
Epoch 4/100
781/781 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.6029 - loss: 0.1034 - val_accuracy: 0.6872 - val_loss: 0.0875 - learning_rate: 1.0000e-04
Epoch 5/100
781/781 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.6260 - loss: 0.0996 - val_accuracy: 0.7010 - val_loss: 0.0843 - learning_rate: 1.0000e-04
Epoch 6/100
781/781 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.6406 - loss: 0.0966 - val_accuracy: 0.7125 - val_loss: 0.0815 - learning_rate: 1.0000e-04
Epoch 7/100
781/781 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - a

In [ ]:
print(TransformerMatcher.model.summary())
TransformerMatcher.save_model(f"{MODEL_DIR}/model.keras")

: 